In [1]:
print("""
@Description: Reshaping and Pivoting
@Author(s): Stephen CUI
@LastEditor(s): Stephen CUI
@CreatedTime: 2023-08-05 16:19:35
""")


@Description: Reshaping and Pivoting
@Author(s): Stephen CUI
@LastEditor(s): Stephen CUI
@CreatedTime: 2023-08-05 16:19:35



# Reshaping and Pivoting

In [2]:
import numpy as np
import pandas as pd

## Reshaping with Hierarchical Indexing

In [3]:
data = pd.DataFrame(np.arange(6).reshape(2, 3), index=pd.Index(['Ohio', 'Colorado'], name='state'),
                    columns=pd.Index(['one', 'two', 'three'], name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [4]:
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [5]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [6]:
result.unstack(level=0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [7]:
result.unstack(level='state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [8]:
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'], dtype='Int64')
s1

a    0
b    1
c    2
d    3
dtype: Int64

In [9]:
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'], dtype='Int64')
s2

c    4
d    5
e    6
dtype: Int64

In [10]:
data2 = pd.concat([s1, s2], keys=['one', 'two'])
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: Int64

In [11]:
data2.unstack()
# unstack() 会引入缺失值

,a,b,c,d,e
one,0,1,2,3,<NA>
two,<NA>,<NA>,4,5,6


In [12]:
# stack() 会默认扔掉缺失的值
data2.unstack().stack()

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: Int64

In [13]:
data2.unstack().stack(dropna=False)

one  a       0
     b       1
     c       2
     d       3
     e    <NA>
two  a    <NA>
     b    <NA>
     c       4
     d       5
     e       6
dtype: Int64

In [14]:
df = pd.DataFrame({'left': result, 'right': result + 5},
                  columns=pd.Index(['left', 'right'], name='side'))
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [15]:
# df.unstack(level=0)
df.unstack(level='state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [16]:
df.unstack(level='state').stack(level='side')

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

## Pivoting “Long” to “Wide” Format

In [17]:
data = pd.read_csv('../examples/macrodata.csv', usecols=['year', 'quarter', 'realgdp', 'infl', 'unemp'])
data.head()

,year,quarter,realgdp,unemp,infl
0,1959,1,2710.349,5.8,0.00
1,1959,2,2778.801,5.1,2.34
2,1959,3,2775.488,5.3,2.74
3,1959,4,2785.204,5.6,0.27
4,1960,1,2847.699,5.2,2.31


In [18]:
periods = pd.PeriodIndex(year=data.pop('year'),
                         quarter=data.pop('quarter'),
                         name='date')
periods

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203)

In [19]:
data.index = periods.to_timestamp("D")
data.head()

,realgdp,unemp,infl
date,,,
1959-01-01,2710.349,5.8,0.00
1959-04-01,2778.801,5.1,2.34
1959-07-01,2775.488,5.3,2.74
1959-10-01,2785.204,5.6,0.27
1960-01-01,2847.699,5.2,2.31


In [20]:
data = data.reindex(columns=['realgdp', 'infl', 'unemp'])
data.columns.name = 'item'

In [21]:
data.head()

item,realgdp,infl,unemp
date,,,
1959-01-01,2710.349,0.00,5.8
1959-04-01,2778.801,2.34,5.1
1959-07-01,2775.488,2.74,5.3
1959-10-01,2785.204,0.27,5.6
1960-01-01,2847.699,2.31,5.2


In [22]:
long_data = (data.stack().reset_index().rename(columns={0: 'value'}))

In [23]:
long_data[:10]

,date,item,value
0,1959-01-01,realgdp,2710.349
1,1959-01-01,infl,0.000
2,1959-01-01,unemp,5.800
3,1959-04-01,realgdp,2778.801
4,1959-04-01,infl,2.340
5,1959-04-01,unemp,5.100
6,1959-07-01,realgdp,2775.488
7,1959-07-01,infl,2.740
8,1959-07-01,unemp,5.300
9,1959-10-01,realgdp,2785.204


In [24]:
pivoted = long_data.pivot(index='date', columns='item', values='value')
pivoted.head()

item,infl,realgdp,unemp
date,,,
1959-01-01,0.00,2710.349,5.8
1959-04-01,2.34,2778.801,5.1
1959-07-01,2.74,2775.488,5.3
1959-10-01,0.27,2785.204,5.6
1960-01-01,2.31,2847.699,5.2


In [25]:
unstacked = long_data.set_index(['date', 'item']).unstack(level='item')
# unstacked['value']
unstacked.head()

value                
item        infl   realgdp unemp
date                            
1959-01-01  0.00  2710.349   5.8
1959-04-01  2.34  2778.801   5.1
1959-07-01  2.74  2775.488   5.3
1959-10-01  0.27  2785.204   5.6
1960-01-01  2.31  2847.699   5.2

## Pivoting “Wide” to “Long” Format

In [26]:
df = pd.DataFrame({"key": ["foo", "bar", "baz"],
                   "A": [1, 2, 3],
                   "B": [4, 5, 6],
                   "C": [7, 8, 9]})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [27]:
melted = pd.melt(df, id_vars='key')
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [28]:
reshaped = melted.pivot(index='key', columns='variable', values='value')
reshaped
# reshaped.columns.name = None

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [29]:
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [30]:
pd.melt(df, id_vars='key', value_vars=['A', 'B'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


In [31]:
pd.melt(df, value_vars=['A', 'B', 'C'])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [32]:
pd.melt(df, value_vars=['key', 'B', 'C'])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9
